In [56]:
import spacy
from datasets import load_dataset
import pandas as pd
import subprocess
import json
import ast
import re

In [33]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
df = pd.DataFrame(ds['corpus'])
df = df.sample(n=10)
text_list = [text for text in df['text']]

NER from lab 6 (baseline):

In [34]:
nlp = spacy.load("pl_core_news_sm")

def get_entities(texts):
    entities = []
    
    for text in texts:
        doc = nlp(text)
        entities += [(ent.lemma_, ent.label_) for ent in doc.ents]

    return entities

In [35]:
spacy_entities = get_entities(text_list)
print("SpaCy entity count: ", len(spacy_entities))

SpaCy entity count:  51


In [36]:
labels = list({item[1] for item in spacy_entities})
labels

['geogName', 'persName', 'placeName', 'date', 'orgName']

In [8]:
import subprocess

def run_ollama(model, text):
    try:
        result = subprocess.run(
            ["ollama", "run", model, text],
            text=True,
            capture_output=True,
            encoding="utf-8",
            check=True
        )
        raw_output = result.stdout
        # print(f"Raw output: {raw_output}")
    
        return raw_output
    except Exception as e:
        print(f"Error: {e}")
        return None

In [9]:
def run_prompts(model_name, prompt, text_list):
    results = []
    for text in text_list:
        result = run_ollama(model_name, prompt.format(text=text))
        results.append(result)
        print(len(results))
    return results

In [41]:
zero_fiqa_results = run_prompts(model_name, zero_shot_prompt_pl, text_list)

1
2
3
4
5
6
7
8
9
10


In [42]:
few_fiqa_results = run_prompts(model_name, few_shot_prompt_pl, text_list)

1
2
3
4
5
6
7
8
9
10


In [71]:
def extract_tuples(text_list):
    extracted_tuples = []
    
    tuple_pattern = r'\(\s*["\'](.*?)["\']\s*,\s*["\'](.*?)["\']\s*\)'
    
    for text in text_list:
        matches = re.findall(tuple_pattern, text)
        extracted_tuples.extend(matches)

    non_empty_tuples = [t for t in extracted_tuples if t[0] != '']
    
    return non_empty_tuples

In [79]:
spacy_entities

[('HSA', 'orgName'),
 ('HSA', 'orgName'),
 ('HRA', 'orgName'),
 ('HSA', 'orgName'),
 ('HSA', 'orgName'),
 ('IRA', 'orgName'),
 ('Roth IRA', 'orgName'),
 ('HSA', 'orgName'),
 ('wypłata', 'persName'),
 ('HSA', 'orgName'),
 ('HSA', 'orgName'),
 ('HSA', 'orgName'),
 ('republikański', 'placeName'),
 ('Bóg', 'persName'),
 ('Amerykanów', 'placeName'),
 ('Trumpa', 'persName'),
 ('ACA', 'orgName'),
 ('Trump', 'persName'),
 ('Kongres', 'persName'),
 ('Trumpa', 'persName'),
 ('Ameryka', 'geogName'),
 ('GTFO', 'persName'),
 ('EOT', 'orgName'),
 ('MG Cranes w Ahmedabad', 'orgName'),
 ('Indie', 'placeName'),
 ('jakby być', 'persName'),
 ('DVR', 'orgName'),
 ('CableCard', 'persName'),
 ('TiVo', 'orgName'),
 ('OnDemand', 'persName'),
 ('gdybyś', 'persName'),
 ('TiVy', 'orgName'),
 ('Verse', 'persName'),
 ('AllVid', 'orgName'),
 ('TiVo', 'orgName'),
 ('TiVo', 'orgName'),
 ('Comcast', 'persName'),
 ('time Warner', 'persName'),
 ('comcast & TW', 'orgName'),
 ('HOA', 'orgName'),
 ('HOA', 'orgName'),
 ('HO

In [72]:
zero_cleaned = extract_tuples(zero_fiqa_results)
zero_cleaned

[('HSA', 'orgName'),
 ('8889', 'date'),
 ('Trump', 'persName'),
 ('ACA', 'orgName'),
 ('50', 'age'),
 ('college', 'placeName'),
 ('CD', 'time'),
 ('funduszu awaryjnego', 'orgName'),
 ('TiVo', 'orgName'),
 ('CableCard', 'orgName'),
 ('FIOS', 'orgName'),
 ('U-Verse', 'orgName'),
 ('AllVid', 'orgName'),
 ('Comcast', 'orgName'),
 ('Time Warner', 'orgName'),
 ('HOA', 'orgName'),
 ('stowarzyszenie mieszkań własnościowych', 'orgName'),
 ('stowarzyszenie condo', 'orgName'),
 ('2015', 'date'),
 ('18 000 USD', 'time'),
 ('5500 USD', 'time')]

In [73]:
few_cleaned = extract_tuples(few_fiqa_results)
few_cleaned

[('HSA', 'orgName'),
 ('FSA', 'orgName'),
 ('HRA', 'orgName'),
 ('Pub. 969', 'date'),
 ('8889', 'date'),
 ('MG Cranes', 'orgName'),
 ('Ahmedabad', 'placeName'),
 ('Uniwersytet Jagielloński', 'orgName'),
 ('Kraków', 'placeName'),
 ('Grunwald', 'placeName'),
 ('15 lipca 1410', 'date'),
 ('Google', 'orgName'),
 ('Warszawa', 'placeName'),
 ('2021', 'date'),
 ('Warszawa', 'placeName'),
 ('Polska', 'geogName'),
 ('2023', 'date'),
 ('Jan Kowalski', 'persName'),
 ('XYZ', 'orgName'),
 ('Uniwersytet Jagielloński', 'orgName'),
 ('Kraków', 'placeName'),
 ('Grunwald', 'placeName'),
 ('15 lipca 1410', 'date'),
 ('Google', 'orgName'),
 ('Warszawa', 'placeName'),
 ('2021', 'date'),
 ('Warszawa', 'placeName'),
 ('Polska', 'geogName'),
 ('2023', 'date'),
 ('Jan Kowalski', 'persName'),
 ('XYZ', 'orgName'),
 ('TiVo', 'orgName'),
 ('CableCard', 'orgName'),
 ('FIOS', 'orgName'),
 ('U-Verse', 'orgName'),
 ('Comcast', 'orgName'),
 ('Time Warner', 'orgName'),
 ('Grunwald', 'placeName'),
 ('Kraków', 'placeName'

In [19]:
model_name = "SpeakLeash/bielik-11b-v2.2-instruct:Q4_K_M"

zero_shot_prompt_pl = """
Wyodrębnij nazwane encje z poniższego tekstu i zaklasyfikuj je do 
jednej z następujących kategorii: ['geogName', 'date', 'placeName', 'persName', 'orgName', 'time']. 
Zwróć wynik jako listę w formacie: [(encja_1, kategotria_1), (encja_2, kategoria_2), ...].
NIE POKAZUJ ŻADNEGO KODU. ZWRÓĆ TYLKO WYNIK. NIE PISZ WYJAŚNIEŃ.

Tekst: {text}"
"""

few_shot_prompt_pl = """
Wyodrębnij nazwane encje z poniższego tekstu i zaklasyfikuj je do 
jednej z następujących kategorii: ['geogName', 'date', 'placeName', 'persName', 'orgName', 'time']. 
Zwróć wynik jako listę w formacie: [(encja_1, kategotria_1), (encja_2, kategoria_2), ...].
NIE POKAZUJ ŻADNEGO KODU. ZWRÓĆ TYLKO WYNIK.

Tekst: {text}"

Przykład 1:
Tekst: "Uniwersytet Jagielloński znajduje się w Krakowie."
Wynik: [("Uniwersytet Jagielloński", "orgName"), ("Kraków", "placeName")]

Przykład 2:
Tekst: "Bitwa pod Grunwaldem odbyła się 15 lipca 1410 roku."
Wynik: [("Grunwald", "placeName"), ("15 lipca 1410", "date")]

Przykład 3:
Tekst: "Google otworzyło nowe biuro w Warszawie w 2021 roku."
Wynik: [("Google", "orgName"), ("Warszawa", "placeName"), ("2021", "date")]
"""

In [80]:
custom_texts = [
    "Warszawa jest stolicą Polski i największym miastem w kraju.",
    "W 2016 roku Polska przyjęła szczyt NATO, który odbył się w Warszawie.",
    "Gdańsk, miasto portowe nad Bałtykiem, jest ważnym ośrodkiem historycznym i kulturalnym.",
    "Nazwa Google pochodzi od terminu googol czyli liczby składającą się z 1 i 100 zer.",
    "Aktualnym kanclerzem Niemiec jest Olaf Scholz, działacz SPD.",
    "Maria Nowak wyjechała do Paryża na konferencję organizowaną przez UNESCO w maju 2023 roku.",
    "Wczoraj odwiedziłem Park Narodowy Białowieski, gdzie spotkałem żubra o imieniu Wojtek.",
    "Niedawno Tesla ogłosiła partnerstwo z firmą Panasonic w celu rozwoju baterii do samochodów elektrycznych.",
    "Amazon planuje otworzyć nowy magazyn w Krakowie do końca przyszłego roku.",
    "Ulubionymi postaciami mojego brata są Hyzio, Dyzio i Zyzio. Kaczor Donald miał się opiekować nimi tymczasowo, dopóki ich ojciec nie wróci ze szpitala, gdzie trafił po jednym z ich wybryków. Rodzice nie wezwali synów do domu, więc Kaczor Donald w 1947 zaadoptował swoich siostrzeńców."
    "Albert Einstein zmarł 18 kwietnia 1955 roku. Był fizykiem-teoretykiem, twórcą szczególnej teorii względności.",
    "W 1865 roku James Clerk Maxwell wysunął przypuszczenie, że światło jest falą elektromagnetyczną. Hipoteza ta została potwierdzona eksperymentalnie w 1889 roku przez Heinricha Hertza, który odkrył fale radiowe.",
    "Wybrzeże Kości Słoniowej to kolejne państwo Afryki Zachodniej, w którym byłe mocarstwo kolonialne zostało zmuszone do zakończenia obecności militarnej. Francja traci wpływy w regionie na rzecz Rosji.",
    "Polski nauczyciel Włodzimierz Bubak został uhonorowany tytułem zdjęcia dnia NASA. Fotografia przedstawia wschód Oriona nad Babią Górą, a wykonana została w noc przesilenia zimowego.",
    "W 2011 roku przywództwo w Korei Północnej przejął Kim Dzong Un.",
    "Polski startup Brainly pozyskał finansowanie od inwestorów z Doliny Krzemowej.",
    "Firma CD Projekt ogłosiła premierę nowej gry w grudniu 2024 roku.",
    "Allegro wprowadziło nową usługę dostawy w ciągu 24 godzin na terenie Polski.",
    "W Katowicach oraz Rudzie Śląskiej Porsche otworzyło nowy salon samochodowy.",
    "Wczoraj poszedłem do sklepu i kupiłem sobie chleb i mleko."
]

ground_truth_one_list = [
    ("Warszawa", "placeName"), ("Polska", "placeName"),
    ("2016", "date"), ("Polska", "placeName"), ("Warszawa", "placeName"),
    ("Gdańsk", "placeName"), ("Bałtyk", "geogName"),
    ("Google", "orgName"),
    ("Olaf Scholz", "persName"), ("Niemcy", "placeName"), ("SPD", "orgName"),
    ("Maria Nowak", "persName"), ("Paryż", "placeName"), ("UNESCO", "orgName"), ("2023", "date"),
    ("Białowieski", "placeName"), ("Wojtek", "persName"),
    ("Tesla", "orgName"), ("Panasonic", "orgName"),
    ("Amazon", "orgName"), ("Kraków", "placeName"), ("2024", "date"),
    ("Hyzio", "persName"), ("Dyzio", "persName"), ("Zyzio", "persName"), ("Kaczor Donald", "persName"), ("1947", "date"),
    ("Albert Einstein", "persName"), ("1955", "date"),
    ("1865", "date"), ("James Clerk Maxwell", "persName"), ("Heinrich Hertz", "persName"),
    ("Wybrzeże Kości Słoniowej", "placeName"), ("Francja", "placeName"), ("Rosja", "placeName"),
    ("Włodzimierz Bubak", "persName"), ("NASA", "orgName"), ("Orion", "geogName"), ("Babia Góra", "geogName"),
    ("2011", "date"), ("Kim Dzong Un", "persName"), ("Korea Północna", "placeName"),
    ("Brainly", "orgName"), ("Dolina Krzemowa", "geogName"),
    ("CD Projekt", "orgName"), ("2024", "date"),
    ("Allegro", "orgName"), ("24 godzin", "time"), ("Polska", "placeName"),
    ("Katowice", "placeName"), ("Ruda Śląska", "placeName"), ("Porsche", "orgName"),
    ("wczoraj", "time")
]


In [82]:
def get_metrics(ground_truth, predictions):
    tp = 0
    fp = 0
    fn = 0

    # eliminujemy case-sensitiveness
    predicted_entities = set((ent[0].lower(), ent[1]) for ent in predictions)
    true_entities = set((ent[0].lower(), ent[1]) for ent in ground_truth)

    tp = len(predicted_entities & true_entities) 
    fp = len(predicted_entities - true_entities)
    fn = len(true_entities - predicted_entities)

    print("TP: ", tp)
    print("FP: ", fp)
    print("FN: ", fn)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    result_dict = {"precision": precision, "recall": recall, "F1-score": f1}
    
    return result_dict

In [14]:
spacy_custom = get_entities(custom_texts)
print("SpaCy entity count: ", len(spacy_custom))

SpaCy entity count:  51


In [15]:
spacy_custom

[('warszawa', 'placeName'),
 ('Polska', 'placeName'),
 ('2016 rok', 'date'),
 ('Polska', 'placeName'),
 ('NATO', 'orgName'),
 ('Warszawa', 'placeName'),
 ('Gdańsk', 'placeName'),
 ('Bałtyk', 'geogName'),
 ('Google', 'orgName'),
 ('Niemcy', 'placeName'),
 ('olaf Scholz', 'persName'),
 ('SPD', 'orgName'),
 ('Maria Nowak', 'persName'),
 ('Paryż', 'placeName'),
 ('UNESCO', 'orgName'),
 ('maj 2023 rok', 'date'),
 ('park narodowy', 'geogName'),
 ('Wojtek', 'persName'),
 ('amazon', 'persName'),
 ('Kraków', 'placeName'),
 ('Hyzio', 'persName'),
 ('Dyzio', 'persName'),
 ('Zyzio', 'persName'),
 ('Donald', 'persName'),
 ('Kaczor Donald', 'persName'),
 ('1947', 'date'),
 ('Albert Einstein', 'persName'),
 ('18 kwiecień 1955 rok', 'date'),
 ('1865 rok', 'date'),
 ('James clerk Maxwell', 'persName'),
 ('1889 rok', 'date'),
 ('Heinrich Hertza', 'persName'),
 ('Afryka Zachodniej', 'geogName'),
 ('Francja', 'placeName'),
 ('Rosja', 'placeName'),
 ('polski', 'placeName'),
 ('Włodzimierz Bubak', 'persName

In [16]:
part1 = custom_texts[:10]
part2 = custom_texts[10:]

In [17]:
part1_results = run_prompts(model_name, zero_shot_prompt_pl, part1)

1
2
3
4
5
6
7
8
9
10


In [20]:
part2_results = run_prompts(model_name, zero_shot_prompt_pl, part2)

1
2
3
4
5
6
7
8
9


In [81]:
combined_results = part1_results + part2_results
cleaned = extract_tuples(combined_results)
print("Bielik entity count: ", len(cleaned))
cleaned

Bielik entity count:  57


[('Warszawa', 'placeName'),
 ('Polska', 'geogName'),
 ('Polska', 'orgName'),
 ('2016', 'date'),
 ('Warszawa', 'placeName'),
 ('Gdańsk', 'placeName'),
 ('Bałtyk', 'geogName'),
 ('Google', 'orgName'),
 ('googol', 'persName'),
 ('Niemiec', 'geogName'),
 ('Olaf Scholz', 'persName'),
 ('SPD', 'orgName'),
 ('Maria Nowak', 'persName'),
 ('Paryża', 'placeName'),
 ('UNESCO', 'orgName'),
 ('maju 2023 roku', 'date'),
 ('Wczoraj', 'date'),
 ('Park Narodowy Białowieski', 'placeName'),
 ('żubra', 'persName'),
 ('Wojtek', 'persName'),
 ('Tesla', 'orgName'),
 ('Panasonic', 'orgName'),
 ('Amazon', 'orgName'),
 ('Kraków', 'geogName'),
 ('Hyzio', 'persName'),
 ('Dyzio', 'persName'),
 ('Zyzio', 'persName'),
 ('Kaczor Donald', 'persName'),
 ('1947', 'date'),
 ('Albert Einstein', 'persName'),
 ('18 kwietnia 1955', 'date'),
 ('geogName', "placeName', 'orgName', 'time"),
 ('1865', 'date'),
 ('1889', 'date'),
 ('James Clerk Maxwell', 'persName'),
 ('Heinrich Hertz', 'persName'),
 ('Wybrzeże Kości Słoniowej', '

In [83]:
spacy_metrics = get_metrics(ground_truth_one_list, spacy_custom)
llm_metrics = get_metrics(ground_truth_one_list, cleaned)

TP:  28
FP:  19
FN:  21
TP:  32
FP:  22
FN:  17


| Model | True Positives (TP) | False Positives (FP) | False Negatives (FN) |
|-------|---------------------|----------------------|----------------------|
| SpaCy | 28                  | 19                   | 21                   |
| LLM   | 32                  | 22                   | 17                   |


### Analyze error patterns and classification mistakes

Analyzing the results presented in the table, we can observe that the errors made by the compared approaches (SpaCy and LLM) are quite similar. In the case of SpaCy, errors were more often related to missing entities that should have been identified. On the other hand, the LLM tended to classify words as entities even when they were not. However, the differences in these metrics are so small that they might be attributed to the specific dataset used rather than reflecting a general trend.

In [84]:
print("SpaCy: ", spacy_metrics)
print("Bielik: ", llm_metrics)

SpaCy:  {'precision': 0.5957446808510638, 'recall': 0.5714285714285714, 'F1-score': 0.5833333333333334}
Bielik:  {'precision': 0.5925925925925926, 'recall': 0.6530612244897959, 'F1-score': 0.6213592233009709}


## Questions (2 points):

### 1. How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?

#### FiQa-pl dataset
The experiment was conducted on 10 texts due to the long response time of the Bielik model (11 billion parameters). On this dataset, SpaCy performed the best. Although it made some errors, it identified the most actual entities. Bielik also performed decently but introduced additional categories, such as "age," even though they were not included in the provided list of allowed categories.

#### Custom dataset

For the 20 custom questions, Bielik outperformed SpaCy in terms of metrics, primarily due to its better lemmatization capabilities. Some examples: 

| Original Form        | SpaCy                | Bielik            |
|----------------------|----------------------|-------------------|
| Heinricha Hertza     | Heinrich Hertza      | Heinrich Hertz    |
| Kim Dzong Un         | kto Dzong Un         | Kim Dzong Un      |
| Doliny Krzemowej     | dolina krzemowy      | Dolina Krzemowa   |


Nevertheless, SpaCy could have performed better than Bielik if not for issues related to lemmatization. I decided not to be case-sensitive when comparing the results

Bielik performs the task quite well, but it has significant limitations. It's necessary to make the right prompts to ensure the model returns only what is needed, without explaining why or providing the code behind the process. Additionally, the response time is much longer for this model. While it is quite large, leading to better results, the trade-off is a longer wait time for answers. It also consumes significantly more resource consuming.

### 2. Which prompting strategy proved most effective for NER and classification tasks? Why?

Zero-shot prompting performed better. The model works slightly faster in that case, and I've noticed it has a better ability to generalize. Few-shot prompts provide some context, which the model sometimes tries to adhere to too much.

### 3. What are the limitations and potential biases of using LLMs for NER and classification?

The main limitations are the response time, resource consumption, and inconsistency of the answers. The model occasionally returns responses that do not have the structure we expect. As a result, we need to write a function that can extract the desired information. However, it's not easy to create a function that will correctly extract the right answer in every case, which means we may occasionally miss some results.

### 4. In what scenarios would you recommend using traditional NER vs. LLM-based approaches?

If the task is not being performed on a large scale and we have some time, I would recommend using an LLM. In case of any errors or doubts, the model can explain why it made a certain decision and can potentially improve based on our feedback.

In other situations, although SpaCy is not perfect, it is good enough in many cases. It is much faster and returns results immediately in the expected format, making the workflow much easier.